# Eigendecomposition of Data and Systems
In this module, we will explore the concept of eigendecomposition, which is a powerful mathematical tool used in various fields, including data science, machine learning, and systems analysis. The key idea underlying eigendecomposition is to let the data speak for itself and to extract the most important features from the data. 

By the end of this module, you will be able to define and demonstrate mastery of the following key concepts:

* __Eigendecomposition__ allows us to decompose a matrix into its constituent parts, the [eigenvectors and eigenvalues](https://en.wikipedia.org/wiki/Eigenvalues_and_eigenvectors). These values help us understand the structure of the data or system represented by the matrix. We'll look at two approaches to estimate the [eigenvalues and eigenvectors](https://en.wikipedia.org/wiki/Eigenvalues_and_eigenvectors) of a matrix:
* __Power iteration method__ estimates the _largest_ eigenvalue/eigenvector pair. Given a _diagonalizable_ matrix $\mathbf{A}$ the power iteration algorithm will produce a number $\lambda$, which is the greatest (in absolute value) eigenvalue of $\mathbf{A}$ and a nonzero vector $\mathbf{v}$ which is a corresponding eigenvector of $\lambda$ such that $\mathbf{A}\mathbf{v} = \lambda\cdot\mathbf{v}$.
* __QR iteration__ is another approach to compute the eigendecomposition of the matrix $\mathbf{A}$. However, unlike power iteration, this approach will give all eigenvalues and eigenvectors of the matrix $\mathbf{A}$. The QR factorization algorithm relies on the [QR decomposition](https://en.wikipedia.org/wiki/QR_decomposition), which itself relies on the [Gram-Schmidt algorithm](https://en.wikipedia.org/wiki/Gram–Schmidt_process).

While we will explore these two approaches to compute the eigendecomposition of a matrix, most, if not all, computing platforms already have built-in functionality to do this computation. Most of the time, there is no need to recreate the wheel (and your implementation will likely be worse in space and time complexity). So always buy when given the chance if the buy option is good.

This is going to be interesting, so let's get started!
___

## Eigendecomposition
Suppose we have a real square matrix $\mathbf{A}\in\mathbb{R}^{m\times{m}}$ which could be a measurement dataset, e.g., the columns of $\mathbf{A}$ represent feature 
vectors $\mathbf{x}_{1},\dots,\mathbf{x}_{m}$ or an adjacency array in a graph with $m$ nodes, etc. Eigenvalue-eigenvector problems involve finding a set of scalar values $\left\{\lambda_{1},\dots,\lambda_{m}\right\}$ called 
[eigenvalues](https://mathworld.wolfram.com/Eigenvalue.html) and a set of linearly independent vectors 
$\left\{\mathbf{v}_{1},\dots,\mathbf{v}_{m}\right\}$ called [eigenvectors](https://mathworld.wolfram.com/Eigenvector.html) such that:
$$
\begin{equation}
\mathbf{A}\;\mathbf{v}_{j} = \lambda_{j}\;\mathbf{v}_{j}\qquad{j=1,2,\dots,m}
\end{equation}
$$
where $\mathbf{v}\in\mathbb{C}^{m}$ and $\lambda\in\mathbb{C}$. We can put the eigenvalues and eigenvectors together in matrix-vector form, which gives us an interesting matrix decomposition:
$$
\mathbf{A} = \mathbf{V}\cdot\text{diag}(\lambda)\cdot\mathbf{V}^{-1}
$$
where $\mathbf{V}$ denotes the matrix of eigenvectors, where the eigenvectors form the columns of the matrix $\mathbf{V}$, $\text{diag}(\lambda)$ denotes a diagonal matrix with the eigenvalues along the main diagonal, and $\mathbf{V}^{-1}$ denotes the inverse of the eigenvalue matrix.

__Symmetric real matricies__

The eigendecomposition of a symmetric real matrix $\mathbf{A}\in\mathbb{R}^{m\times{m}}$ has some special properties. 
First, all the eigenvalues $\left\{\lambda_{1},\lambda_{2},\dots,\lambda_{m}\right\}$ of the matrix $\mathbf{A}$ are real-valued.
Next, the eigenvectors $\left\{\mathbf{v}_{1},\mathbf{v}_{2},\dots,\mathbf{v}_{m}\right\}$ of the matrix $\mathbf{A}$ are orthogonal, i.e., $\left<\mathbf{v}_{i},\mathbf{v}_{j}\right> = 0$ for $i\neq{j}$. Finally, the (normalized) eigenvectors $\mathbf{v}_{j}/\lVert|{\mathbf{v}_{j}}\rVert|$ of a symmetric real-valued matrix 
form an orthonormal basis for the space spanned by the matrix $\mathbf{A}$ such that:
$$
\begin{equation}
\left<\hat{\mathbf{v}}_{i},\hat{\mathbf{v}}_{j}\right> = \delta_{ij}\qquad\text{for}\quad{i,j\in\mathbf{A}}
\end{equation}
$$
where $\delta_{ij}$ is the [Kronecker delta function](https://en.wikipedia.org/wiki/Kronecker_delta). 

__So, why is this interesting__?
* Eigenvectors represent fundamental directions of the matrix $\mathbf{A}$. For the linear transformation defined by a matrix $\mathbf{A}$, eigenvectors are the only vectors that do not change direction during the transformation. If we think about the matrix $\mathbf{A}$ as a machine, we put the eigenvector $\mathbf{v}_{\star}$ into the machine, and we get back the same eigenvector $\mathbf{v}_{\star}$ multiplied by a scalar, the eigenvalue $\lambda_{\star}$.
* Eigenvalues are scale factors for their eigenvector. An eigenvalue is a scalar that indicates how much a corresponding eigenvector is stretched or compressed during a linear transformation represented by the matrix $\mathbf{A}$.
* We can use the eigendecomposition to diagonalize the matrix $\mathbf{A}$, i.e., transform the matrix into a diagonal form where the eigenvalues lie along the main diagonal. To see this, solve the eigendecomposition for the $\text{diag}(\lambda) = \mathbf{V}^{-1}\cdot\mathbf{A}\cdot\mathbf{V}$. We can also use the eigenvalues to classify a matrix $\mathbf{A}$ as positive (semi)definite or negative (semi)definite (which will be handy later).
* If the matrix $\mathbf{A}$ is symmetric, and all entries are positive, then all the eigenvalues will be real-valued, and the eigenvectors will be orthogonal (handy properties, as we shall soon see).

Finally, another interpretation we'll also explore later is that eigenvectors represent the most critical directions in the data or system, and eigenvalues (or functions of them) represent their importance. Hmm, interesting. But how can we calculate them (given the buy versus build caveat above)?

___

### Method 1: Power iteration
The [power iteration method](https://en.wikipedia.org/wiki/Power_iteration) is an iterative algorithm to compute the largest eigenvalue and its corresponding eigenvector of a square (real) matrix; we'll consider only real-valued matrices here, but this approach can also be used for matrices with complex entries. 

* _Application:_ Perhaps the most famous application of [power iteration](https://en.wikipedia.org/wiki/Power_iteration) is the [Google PageRank algorithm](https://epubs.siam.org/doi/10.1137/050623280). Google's PageRank algorithm, which uses power iteration, utilizes the dominant eigenvalue and its corresponding eigenvector of a link connection matrix to assess the importance of web pages within a connection network.

How does the [power iteration method](https://en.wikipedia.org/wiki/Power_iteration) work?

__Phase 1: Eigenvector__: Suppose we have a real-valued square _diagonalizable_ matrix $\mathbf{A}\in\mathbb{R}^{m\times{m}}$ whose eigenvalues have the property $|\lambda_{1}|\geq|\lambda_{2}|\dots\geq|\lambda_{m}|$. Then, the eigenvector $\mathbf{v}_{1}\in\mathbb{C}^{m}$ which corresponds to the largest eigenvalue $\lambda_{1}\in\mathbb{C}$ can be (iteratively) estimated as:
$$
\mathbf{v}_{1}^{(k+1)} = \frac{\mathbf{A}\mathbf{v}_{1}^{(k)}}{\Vert \mathbf{A}\mathbf{v}_{1}^{(k)} \Vert}\quad{k=0,1,2\dots}
$$

where $\lVert \star \rVert$ denotes the [L2 (Euclidean) vector norm](https://mathworld.wolfram.com/L2-Norm.html). The [power iteration method](https://en.wikipedia.org/wiki/Power_iteration) converges to a value for the eigenvector as $k\rightarrow\infty$ when a few properties are true, namely, $|\lambda_{1}|/|\lambda_{2}| < 1$ (which is unknown beforehand), and we pick an appropriate initial guess for $\mathbf{v}_{1}$ (in our case, a random vector will work).

__Phase 2: Eigenvalue__: Once we have an estimate for the eigenvector $\hat{\mathbf{v}}_{1}$, we can estimate the corresponding eigenvalue $\hat{\lambda}_{1}$ using [the Rayleigh quotient](https://en.wikipedia.org/wiki/Rayleigh_quotient). This argument proceeds from the definition of the eigenvalues and eigenvectors. We know, from the definition of eigenvalue-eigenvector pairs, that:
$$
\mathbf{A}\hat{\mathbf{v}}_{1} - \hat{\lambda}_{1}\hat{\mathbf{v}}_{1}\simeq{0}
$$
where we use the $\simeq$ symbol because we don't have the true eigenvector $\mathbf{v}_{1}$, only an estimate of it. To solve this expression for the (estimated) eigenvalue $\hat{\lambda}_{1}$, we multiply through by the transpose of the eigenvector and solve for the eigenvalue:
$$
\hat{\lambda}_{1} \simeq \frac{\hat{\mathbf{v}}_{1}^{\top}\mathbf{A}\hat{\mathbf{v}}_{1}}{\hat{\mathbf{v}}_{1}^{\top}\hat{\mathbf{v}}_{1}} = \frac{\left<\mathbf{A}\hat{\mathbf{v}}_{1},\hat{\mathbf{v}}_{1}\right>}{\left<\hat{\mathbf{v}}_{1},\hat{\mathbf{v}}_{1}\right>}
$$
where $\left<\star,\star\right>$ denotes [an inner product](https://mathworld.wolfram.com/InnerProduct.html). 

Let's explore a pseudo-code implementation of the [power iteration method](https://en.wikipedia.org/wiki/Power_iteration) to estimate the largest eigenvalue and its corresponding eigenvector of a matrix $\mathbf{A}$.

__Initialization__: Given a real-valued square matrix $\mathbf{A}\in\mathbb{R}^{m\times{m}}$, we specify $\epsilon>0$ as the convergence tolerance, `maxiter` as the maximum number of iterations, an initial guess for the eigenvector $\mathbf{v}_{1}^{(0)}$ (we'll use a random vector), an iteration counter $k\gets{1}$, and a boolean variable $\texttt{converged}\gets{\texttt{false}}$ to indicate whether the algorithm has converged.

Normalize the initial guess for the eigenvector: $\mathbf{v}_{1}^{(0)} = \frac{\mathbf{v}_{1}^{(0)}}{\lVert \mathbf{v}_{1}^{(0)} \rVert}$.

While not $\texttt{converged}$ __do__:
- Compute the matrix-vector product $\mathbf{y}^{(k)}\gets\mathbf{A}\mathbf{v}_{1}^{(k-1)}$.
- Normalize the vector $\mathbf{v}_{1}^{(k)}\gets\frac{\mathbf{y}^{(k)}}{\lVert \mathbf{y}^{(k)} \rVert}$.
- Compute the Rayleigh quotient to estimate the eigenvalue: $\hat{\lambda}_{1}^{(k)} \gets  {\mathbf{v}_{1}^{(k)\top}\mathbf{A}\mathbf{v}_{1}^{(k)}}/{\mathbf{v}_{1}^{(k)\top}\mathbf{v}_{1}^{(k)}}$.
- Check for convergence:
   1. If $\lVert \mathbf{v}_{1}^{(k)} - \mathbf{v}_{1}^{(k-1)} \rVert\leq\epsilon$, then set $\texttt{converged}\gets{\texttt{true}}$ and return $\hat{\lambda}_{1} = \hat{\lambda}_{1}^{(k)}$ and $\hat{\mathbf{v}}_{1} = \frac{\mathbf{v}_{1}^{(k)}}{\lVert \mathbf{v}_{1}^{(k)} \rVert}$.
   2. If $k\geq\texttt{maxiter}$, then set $\texttt{converged}\gets{\texttt{true}}$ to terminate the algorithm, return the last estimate of the eigenvalue and eigenvector (optionally flagging that convergence tolerance was not met).
   3. Increment $k\gets{k+1}$.


While simple to implement, the [power iteration method](https://en.wikipedia.org/wiki/Power_iteration) may exhibit slow convergence, mainly when the largest eigenvalue is close in magnitude to other eigenvalues, i.e., $|\lambda_{1}|/|\lambda_{2}| \sim 1$.

___

### Method 2: QR Iteration
[QR iteration](https://en.wikipedia.org/wiki/QR_algorithm) is a fundamental technique in numerical linear algebra, primarily used for computing the eigenvalues and eigenvectors of matrices. The algorithm leverages the concept of [QR decomposition](https://en.wikipedia.org/wiki/QR_decomposition), which expresses a (rectangular) matrix $\mathbf{A}\in\mathbb{R}^{n\times{m}}$ as a product of an orthogonal matrix $\mathbf{Q}\in\mathbb{R}^{n\times{n}}$ and an upper triangular matrix $\mathbf{R}\in\mathbb{R}^{n\times{m}}$:
$$
\mathbf{A} = \mathbf{Q}\mathbf{R}
$$
where $\mathbf{Q}^{\top}\mathbf{Q} = \mathbf{I}$. The core of the QR iteration algorithm involves iteratively decomposing a given matrix $\mathbf{A}$ into its $\mathbf{Q}$ and $\mathbf{R}$ factors and then reformulating the matrix for subsequent iterations. Under certain conditions, [QR iteration](https://en.wikipedia.org/wiki/QR_algorithm) will converge to a triangular matrix with the eigenvalues of the original matrix $\mathbf{A}$ listed on the diagonal.

__Algorithm__
* __Initialization__. We begin by specifying an initial matrix $\mathbf{A}_{1} = \mathbf{A}$, the maximum number of iterations `maxiter` that we are willing to do, and a tolerance parameter $\epsilon$.
* __Update__. For iteration $k = 1,2,\dots$, compute the [QR decomposition](https://en.wikipedia.org/wiki/QR_decomposition) of $\mathbf{A}_{k} = \mathbf{Q}_{k}\mathbf{R}_{k}$. We then form a new matrix $\mathbf{A}_{k+1} = \mathbf{R}_{k}\mathbf{Q}_{k}$, which can be re-written as $\mathbf{A}_{k+1} = \mathbf{Q}^{T}_{k}\mathbf{A}_{k}\mathbf{Q}_{k}$.
* __Stopping__. We stop the iteration procedure after `maxiter` iterations is reached or when the difference between successive iterations is _small_ in some sense, i.e., $\lVert \mathbf{A}_{k+1} - \mathbf{A}_{k} \rVert_{1}\leq\epsilon$ where $\lVert\star\rVert_{1}$ denotes the [p = 1 matrix norm](https://en.wikipedia.org/wiki/Matrix_norm), or perhaps $\lVert \lambda_{k+1} - \mathbf{\lambda}_{k} \rVert_{2}\leq\epsilon$ where $\lVert\star\rVert_{2}$ is [the L2-vector norm](https://en.wikipedia.org/wiki/Norm_(mathematics)#Euclidean_norm), i.e., the eigenvalues don't change between iterations, and $\epsilon$ is a tolerance parameter.

Once we have converged to matrix $\mathbf{A}_{\star}$, we get the eigenvalue from the diagonal of $\mathbf{A}_{\star}$. To compute the eigenvectors, we solve the homogenous system of linear algebraic equations:
$$
\left(\mathbf{A} - \lambda_{\star}\mathbf{I} \right)\cdot\mathbf{v}_{\star} = \mathbf{0}
$$

Before we implement [QR iteration](https://en.wikipedia.org/wiki/QR_algorithm), let's look at how to compute the $\mathbf{Q}$ and $\mathbf{R}$ matrices.

### Classical and Modified Gram-Schmidt
The [QR decomposition](https://en.wikipedia.org/wiki/QR_decomposition) can be computed using a variety of approaches, including a handy technique called [the Gram–Schmidt algorithm](https://en.wikipedia.org/wiki/Gram%E2%80%93Schmidt_process). In principle, Gram-Schmidt orthogonalization generates a set of mutually orthogonal vectors $\mathbf{q}_{1},\mathbf{q}_{2},\dots, \mathbf{q}_{n}$ starting from a set of linearly independent vectors $\mathbf{x}_{1},\mathbf{x}_{2},\dots,\mathbf{x}_{n}$ 
by subtracting the projection of each vector onto the previous vectors, i.e.,
$$
\begin{equation}
\mathbf{q}_{k}=\mathbf{x}_{k}-\sum_{i=1}^{k-1}c_{k,i}\cdot\mathbf{q}_{i},
\qquad{k=1,\dots,n}
\end{equation}
$$
where the coefficients $c_{k,1},c_{k,2},\dots,c_{k,k-1}$ are chosen to make the vectors $\mathbf{q}_{1},\mathbf{q}_{2},\dots,\mathbf{q}_{k}$ orthogonal.
The $c_{\star}$ coefficients represent the component of the vector $\mathbf{x}_{k}$ that lies in the direction of the vectors $\mathbf{q}_{1},\mathbf{q}_{2},\dots,\mathbf{q}_{k-1}$. See [the course notes for details about computing $c_{\star}$](https://github.com/varnerlab/CHEME-5820-Lectures-Spring-2025/blob/main/lectures/week-2/L2a/docs/Notes.pdf).






Classical Gram-Schmidt can sometimes produce _almost_ orthogonal vectors because of roundoff error, which led to the Modified Gram-Schmidt algorithm. Check out the [Classical and Modified Gram-Schmidt pseudo-code in the course notes](https://github.com/varnerlab/CHEME-5820-Lectures-Spring-2025/blob/main/lectures/week-2/L2a/docs/Notes.pdf).

__Additional references__:
* [Prof. David Bindel: Cornell CS 6210: Matrix Computations (2016), Lecture on Power Iteration](https://www.cs.cornell.edu/~bindel/class/cs6210-f16/lec/2016-10-17.pdf)
* [Prof. Tom Trogdon: UCI MATH 105A: Numerical Analysis (2016), Lecture 22: The Power Method](https://faculty.washington.edu/trogdon/105A/html/Lecture22.html)
* [Prof. Thomas Strohmer: UCD MATH 108: Mathematical Algorithms for Artificial Intelligence and Big Data (2017), Lecture on PageRank](https://math.ucdavis.edu/%7Estrohmer/courses/180BigData/180lecture_pagerank.pdf)